# <center>Structural Analysis and Visualization of Networks</center>

## <center>Home Assignment #4: Community Detection Algorithms

### <center>Student: *Maxim Ushakov*</center>

#### <hr /> General Information

**Due Date:** 03.04.2016 23:59 <br \>
**Late submission policy:** the task will not be graded! <br \>


Please send your reports to <network.hse.2016@gmail.com> with message subject of the following structure:<br \> **[HSE Networks 2015] *{LastName}* *{First Name}* HA*{Number}***

Support your computations with figures and comments. <br \>
If you are using IPython Notebook you may use this file as a starting point of your report.<br \>
<br \>
<hr \>

## Problems

### Task 1* (For those who have not done that during the seminar)

On this seminar your are asked to implement simple community detection algorightm. It is called [Markov Cluster Algorithm](http://micans.org/mcl/) (MCL).

Implement Markor Clustering Algorithm

**Input:** Transition matrix $T = D^{-1}A$

**Output:** Adjacency matrix $M^*$

1. Set $M = T$
2. **repeat:**
    3. *Expansion Step:* $M = M^p$ (usually $p=2$)
    4. *Inflation Step:* Raise every entry of $M$ to the power $\alpha$ (usualy $\alpha=2$)
    5. *Renormalize:* Normalize each row by its sum
    6. *Prunning:* Replace entries that are close to $0$ by pure $0$
7. **until** $M$ converges
8. $M^* = M$
<br\>
<br\>

As a result you should get a cluster matrix s.t. elements of the cluster correspont to nonzero elements of the columns of the matrix. 
<br\>
* Run this method for network [1](https://www.hse.ru/data/2016/03/15/1127695811/network1.mat), [2](https://www.hse.ru/data/2016/03/15/1127699956/network2.mat) and [3](https://www.hse.ru/data/2016/03/15/1127703057/network3.mat).
* Play with the parameters ($p$, $\alpha$, zero tolerance), analyse the results

<hr />


In [177]:
import scipy.io
import numpy as np
import scipy.sparse as sparse
import numpy.linalg as linalg

#read the data
data1 = scipy.io.loadmat('network1.mat')
A1 = data1['A'].astype('float')
label1 = data1['Comm']
data2 = scipy.io.loadmat('network2.mat')
A2 = data2['A'].astype('float')
label2 = data2['Comm']
data3 = scipy.io.loadmat('network3.mat')
A3 = data3['A'].astype('float')
label3 = data3['Comm']

In [196]:
def MCL(A, p, alpha, tol):
    step = 1
    eps2 = 0.0001
    col_sums = A.sum(axis = 0)
    T = A / col_sums[np.newaxis, :]
    M = T
    while step <= 100:
        #print('step ', step)
        step += 1
        # Expancion step:
        M1 = np.linalg.matrix_power(M, p)
        # Inflation step:
        M1 = np.power(M1, alpha)
        col_sums = M1.sum(axis = 0)
        M1 = M1 / col_sums[np.newaxis, :]
        M1[M1<=tol] = 0
        if np.linalg.norm(M - M1) < eps2:
            return M1, step
        else:
            M = M1.copy()  

In [213]:
def check_result(M, label):
    accuracy = 0.0
    
    #denominator
    den = 0
    for i in range(len(label)):
        for j in range(len(label)):
            if M[i,j] > 0 and label[i] == label[j]:
                accuracy += 1
            if M[i,j] > 0:
                den += 1
    accuracy /= den
    return accuracy

In [214]:
p_mass = [2, 4, 8]
alpha_mass = [2, 4, 8]
eps_mass = [0.005, 0.001]

for p in p_mass:
    for alpha in alpha_mass:
        for eps in eps_mass:
            M1, num_steps1 = MCL(A1, p, alpha, eps)
            acc1 = check_result(M1, label1)
            M2, num_steps2 = MCL(A2, p, alpha, eps)
            acc2 = check_result(M2, label2)
            M3, num_steps3 = MCL(A3, p, alpha, eps)
            acc3 = check_result(M3, label3)
            print('p=%d, alpha=%f, eps=%f, av_acc=%f, av_num_step=%f' % (p, alpha, eps, (acc1+acc2+acc3)/3, (num_steps1+num_steps2+num_steps3)/3))
            

p=2, alpha=2.000000, eps=0.005000, av_acc=1.000000, av_num_step=14.000000
p=2, alpha=2.000000, eps=0.001000, av_acc=1.000000, av_num_step=16.000000
p=2, alpha=4.000000, eps=0.005000, av_acc=1.000000, av_num_step=5.000000
p=2, alpha=4.000000, eps=0.001000, av_acc=1.000000, av_num_step=5.000000
p=2, alpha=8.000000, eps=0.005000, av_acc=1.000000, av_num_step=4.000000
p=2, alpha=8.000000, eps=0.001000, av_acc=1.000000, av_num_step=4.000000
p=4, alpha=2.000000, eps=0.005000, av_acc=0.500000, av_num_step=16.333333
p=4, alpha=2.000000, eps=0.001000, av_acc=0.500000, av_num_step=16.333333
p=4, alpha=4.000000, eps=0.005000, av_acc=0.750000, av_num_step=8.666667
p=4, alpha=4.000000, eps=0.001000, av_acc=0.750000, av_num_step=9.000000
p=4, alpha=8.000000, eps=0.005000, av_acc=1.000000, av_num_step=7.000000
p=4, alpha=8.000000, eps=0.001000, av_acc=0.992188, av_num_step=7.000000
p=8, alpha=2.000000, eps=0.005000, av_acc=0.500000, av_num_step=15.000000
p=8, alpha=2.000000, eps=0.001000, av_acc=0.25

As we can see from results, the algorithm works better with small $p$ ($p\ge 2$). Increasing of $\alpha$ doesn't influence the result significantly, however, $\frac{\alpha}{p}$ should be great for accurate results. Also increasement of $\alpha$ reduces the number of steps. It seems that zero tolerance parameter $\varepsilon$ doesn't impact on results significantly, but too small values of $\varepsilon$ may lead to a little unaccuracy and increasement of steps to proceed, but large values of zero tolerance may lead to mistakes, or even bugs (if $\varepsilon > 1$, then matrix will be turn to $0$ on the first step).   

### Task 2

Load [Yahoo Music network](https://www.hse.ru/data/2016/03/15/1127704844/music_data.mat). Edges in this network appear if enough number of users have given ratings to both music bands. Note, that edges are weighted with similarity of the ratings.

* Implement *multilevel spectral recursive partitioning* algorithm that was described during the lecture
* Visualize community structure of the network and output some of the dense clusters (with interpretation, if you can)

You can load .mat files with the following commands:

In [232]:
import scipy.io
import scipy.sparse.linalg as slinalg

data = scipy.io.loadmat('music_data.mat')
artists = data['artists']
A = data['A'].astype('float')

In [246]:
def MSRP(A, num_clusters):
    
    D = sparse.csr_matrix(np.diagflat(A.sum(1)))                     
    L = D - A                                       
    eig_val, eig_vec = slinalg.eigsh(L, 2, D, which = 'SM')
    s = [(i, eig_vec[i,1]) for i in range(len(eig_vec[:,1]))]
    s = sorted(s, key=lambda v: v[1], reverse = False)
    
    d = [(i, s[i+1][1]-s[i][1]) for i in range(len(s)-1)]
    d = sorted(d, key=lambda v: v[1], reverse = True)
    diff_ids = [v[0] for v in d[0:num_clusters]].append(0)
    print(diff_ids)
    diff_ids = sorted(diff_ids)
    
    cl = []
    cl_id = 0
    for bind in range(len(1, diff_ids)):
        cl_id += 1
        cl.extend([(v[0], cl_id) for v in s[diff_ids[bind-1]:(diff_ids[bind]+1)]])
    
    return cl    

In [247]:
cl = MSRP(A, 4)

None


TypeError: 'NoneType' object is not iterable